In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
import seaborn as sns

# OUTIL  IBM

## pour les Volumes

In [2]:
# création de la liste des fichiers à récupérer pour IBM volume

basepath_ibm_volume = 'C:/Users/lsarlat/Documents/Capacity_planning/CP3/données_version3/ibm/volume'
# List all files in a directory using scandir()
liste_ibm_volume=[]
with os.scandir(basepath_ibm_volume) as entries: # renvoie un itérateur
    for entry in entries:
        if entry.is_file():
            liste_ibm_volume.append(str(entry.name))
print(len(liste_ibm_volume))

773


In [105]:
# création du DF principal à partir de la liste_IBM_volume
df_ibm_volume = pd.DataFrame()
for name in liste_ibm_volume:
    with open(os.path.join(basepath_ibm_volume, name)) as f:
        df_temp = pd.read_fwf(f)
        df_temp['date']= datetime.datetime.strptime(str(int(name[:8])),'%Y%m%d')
        df_ibm_volume = pd.concat([df_ibm_volume, df_temp])
df_ibm_volume

,vdisk_id,vdisk_name,copy_id,mdisk_grp_id,mdisk_grp_name,capacity,used_capacity,real_capacity,free_capacity,overallocation,...,grainsize,se_copy,compressed_copy,uncompressed_used_capacity,parent_mdisk_grp_id,parent_mdisk_grp_name,encrypt,deduplicated_copy,used_capacity_before_reduction,date
0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,0,3,SDC3_T0,10.00GB,9.83GB,10.01GB,187.30MB,99,...,256.0,yes,no,9.83GB,3,SDC3_T0,yes,no,NaN,2018-10-03
1,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,1,2,SDC1_T0,10.00GB,9.83GB,10.01GB,184.75MB,99,...,256.0,yes,no,9.83GB,2,SDC1_T0,yes,no,NaN,2018-10-03
2,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,1,0,SDC1_T2,200.00GB,44.46GB,48.48GB,4.02GB,412,...,NaN,no,yes,157.05GB,0,SDC1_T2,yes,no,NaN,2018-10-03
3,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,1,0,SDC1_T2,200.00GB,44.47GB,48.50GB,4.03GB,412,...,NaN,no,yes,157.05GB,0,SDC1_T2,yes,no,NaN,2018-10-03
4,3,INFS_19DC1_T2M_TEST_EXLIDAT1_01,0,1,SDC3_T2,10.00GB,10.53GB,10.53GB,1.44MB,94,...,NaN,no,yes,9.82GB,1,SDC3_T2,yes,no,NaN,2018-10-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,753,PRVN_19DC1_T2M_PRVN_01_DS_01,1,3,SDC3_T2,512.00GB,86.44GB,96.68GB,10.24GB,529,...,NaN,no,yes,188.27GB,3,SDC3_T2,yes,no,NaN,2020-11-13
909,754,PRVN_19DC3_T2M_PRVN_01_DS_01,0,3,SDC3_T2,512.00GB,27.94GB,38.19GB,10.25GB,1340,...,NaN,no,yes,72.43GB,3,SDC3_T2,yes,no,NaN,2020-11-13
910,754,PRVN_19DC3_T2M_PRVN_01_DS_01,1,9,SDC1_T2,512.00GB,27.85GB,38.10GB,10.25GB,1343,...,NaN,no,yes,72.43GB,9,SDC1_T2,yes,no,NaN,2020-11-13
911,761,SIGN_19DC1_T2M_BACKUP_01,0,6,SDC1_T2_FCM,1.00TB,945.09GB,965.57GB,20.48GB,106,...,256.0,yes,no,945.09GB,6,SDC1_T2_FCM,yes,no,NaN,2020-11-13


In [106]:
#suppression des lignes où la used_capcity est null
df_ibm_volume = df_ibm_volume[df_ibm_volume.used_capacity.notnull()]

In [107]:
# création d'une fonction qui recalcule les valeurs en MB et ne fait rien si la valeur null
import math
def en_MB(x):
    if type(x)==str:
        if "GB" in x :
            x = x.replace("GB","")
            x= float(x)*1024
        elif "G" in x:
            x = x.replace("G","")
            x= float(x)*1024
        elif "MB" in x:
            x = x.replace("MB","")
            x= float(x)
        elif "TB" in x:
            x = x.replace("TB","")
            x = float(x)*1024*1024
    elif math.isnan(float(x)):
        pass
    return x

In [108]:
# transformation des colonnes capacity, used_capacity,real_capacity, free_capacity, uncompressed_used_capacity, used_capacity_before_reduction
for colonne in ["capacity", "used_capacity","real_capacity", "free_capacity", "uncompressed_used_capacity", "used_capacity_before_reduction"]:
    df_ibm_volume[colonne] = df_ibm_volume[colonne].map(lambda x : en_MB(x) )
df_ibm_volume

C:\Users\lsarlat\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,vdisk_id,vdisk_name,copy_id,mdisk_grp_id,mdisk_grp_name,capacity,used_capacity,real_capacity,free_capacity,overallocation,...,grainsize,se_copy,compressed_copy,uncompressed_used_capacity,parent_mdisk_grp_id,parent_mdisk_grp_name,encrypt,deduplicated_copy,used_capacity_before_reduction,date
0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,0,3,SDC3_T0,10240.0,10065.92,10250.24,187.30,99,...,256.0,yes,no,10065.92,3,SDC3_T0,yes,no,NaN,2018-10-03
1,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,1,2,SDC1_T0,10240.0,10065.92,10250.24,184.75,99,...,256.0,yes,no,10065.92,2,SDC1_T0,yes,no,NaN,2018-10-03
2,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,1,0,SDC1_T2,204800.0,45527.04,49643.52,4116.48,412,...,NaN,no,yes,160819.20,0,SDC1_T2,yes,no,NaN,2018-10-03
3,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,1,0,SDC1_T2,204800.0,45537.28,49664.00,4126.72,412,...,NaN,no,yes,160819.20,0,SDC1_T2,yes,no,NaN,2018-10-03
4,3,INFS_19DC1_T2M_TEST_EXLIDAT1_01,0,1,SDC3_T2,10240.0,10782.72,10782.72,1.44,94,...,NaN,no,yes,10055.68,1,SDC3_T2,yes,no,NaN,2018-10-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,753,PRVN_19DC1_T2M_PRVN_01_DS_01,1,3,SDC3_T2,524288.0,88514.56,99000.32,10485.76,529,...,NaN,no,yes,192788.48,3,SDC3_T2,yes,no,NaN,2020-11-13
909,754,PRVN_19DC3_T2M_PRVN_01_DS_01,0,3,SDC3_T2,524288.0,28610.56,39106.56,10496.00,1340,...,NaN,no,yes,74168.32,3,SDC3_T2,yes,no,NaN,2020-11-13
910,754,PRVN_19DC3_T2M_PRVN_01_DS_01,1,9,SDC1_T2,524288.0,28518.40,39014.40,10496.00,1343,...,NaN,no,yes,74168.32,9,SDC1_T2,yes,no,NaN,2020-11-13
911,761,SIGN_19DC1_T2M_BACKUP_01,0,6,SDC1_T2_FCM,1048576.0,967772.16,988743.68,20971.52,106,...,256.0,yes,no,967772.16,6,SDC1_T2_FCM,yes,no,NaN,2020-11-13


In [109]:
# renommage des colonnes
df_ibm_volume = df_ibm_volume.rename(columns={"vdisk_id":"ID","vdisk_name":"name","mdisk_grp_name": "pool"})

#création d'une colonne offre
df_ibm_volume['offre'] = df_ibm_volume.pool
df_ibm_volume.offre= df_ibm_volume.offre.map(lambda x: x[5:])

# création d'une colonne origine
df_ibm_volume["origine"] = np.array("ibm")
df_ibm_volume

,ID,name,copy_id,mdisk_grp_id,pool,capacity,used_capacity,real_capacity,free_capacity,overallocation,...,compressed_copy,uncompressed_used_capacity,parent_mdisk_grp_id,parent_mdisk_grp_name,encrypt,deduplicated_copy,used_capacity_before_reduction,date,offre,origine
0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,0,3,SDC3_T0,10240.0,10065.92,10250.24,187.30,99,...,no,10065.92,3,SDC3_T0,yes,no,NaN,2018-10-03,T0,ibm
1,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,1,2,SDC1_T0,10240.0,10065.92,10250.24,184.75,99,...,no,10065.92,2,SDC1_T0,yes,no,NaN,2018-10-03,T0,ibm
2,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,1,0,SDC1_T2,204800.0,45527.04,49643.52,4116.48,412,...,yes,160819.20,0,SDC1_T2,yes,no,NaN,2018-10-03,T2,ibm
3,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,1,0,SDC1_T2,204800.0,45537.28,49664.00,4126.72,412,...,yes,160819.20,0,SDC1_T2,yes,no,NaN,2018-10-03,T2,ibm
4,3,INFS_19DC1_T2M_TEST_EXLIDAT1_01,0,1,SDC3_T2,10240.0,10782.72,10782.72,1.44,94,...,yes,10055.68,1,SDC3_T2,yes,no,NaN,2018-10-03,T2,ibm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,753,PRVN_19DC1_T2M_PRVN_01_DS_01,1,3,SDC3_T2,524288.0,88514.56,99000.32,10485.76,529,...,yes,192788.48,3,SDC3_T2,yes,no,NaN,2020-11-13,T2,ibm
909,754,PRVN_19DC3_T2M_PRVN_01_DS_01,0,3,SDC3_T2,524288.0,28610.56,39106.56,10496.00,1340,...,yes,74168.32,3,SDC3_T2,yes,no,NaN,2020-11-13,T2,ibm
910,754,PRVN_19DC3_T2M_PRVN_01_DS_01,1,9,SDC1_T2,524288.0,28518.40,39014.40,10496.00,1343,...,yes,74168.32,9,SDC1_T2,yes,no,NaN,2020-11-13,T2,ibm
911,761,SIGN_19DC1_T2M_BACKUP_01,0,6,SDC1_T2_FCM,1048576.0,967772.16,988743.68,20971.52,106,...,no,967772.16,6,SDC1_T2_FCM,yes,no,NaN,2020-11-13,T2_FCM,ibm


In [110]:
#nombre de volumes différents
len(df_ibm_volume.name.unique())

1249

In [111]:
df_ibm_volume.overallocation.describe()

count    518917.000000
mean       1133.140866
std        3812.274236
min          94.000000
25%         138.000000
50%         288.000000
75%         875.000000
max      365070.000000
Name: overallocation, dtype: float64

In [112]:
df_ibm_volume.compressed_copy.value_counts()

no     310992
yes    207925
Name: compressed_copy, dtype: int64

In [113]:
df_ibm_volume.encrypt.value_counts()

yes    518917
Name: encrypt, dtype: int64

In [114]:
df_ibm_volume.deduplicated_copy.value_counts()

no    518917
Name: deduplicated_copy, dtype: int64

In [115]:
df_ibm_volume_reduit = df_ibm_volume[["ID","name","pool","offre","capacity","used_capacity",'date','origine']].copy()

# réinitialise les index
df_ibm_volume_reduit.reset_index(drop=True,inplace =True)
df_ibm_volume_reduit

,ID,name,pool,offre,capacity,used_capacity,date,origine
0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC3_T0,T0,10240.0,10065.92,2018-10-03,ibm
1,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC1_T0,T0,10240.0,10065.92,2018-10-03,ibm
2,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,SDC1_T2,T2,204800.0,45527.04,2018-10-03,ibm
3,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,SDC1_T2,T2,204800.0,45537.28,2018-10-03,ibm
4,3,INFS_19DC1_T2M_TEST_EXLIDAT1_01,SDC3_T2,T2,10240.0,10782.72,2018-10-03,ibm
...,...,...,...,...,...,...,...,...
518912,753,PRVN_19DC1_T2M_PRVN_01_DS_01,SDC3_T2,T2,524288.0,88514.56,2020-11-13,ibm
518913,754,PRVN_19DC3_T2M_PRVN_01_DS_01,SDC3_T2,T2,524288.0,28610.56,2020-11-13,ibm
518914,754,PRVN_19DC3_T2M_PRVN_01_DS_01,SDC1_T2,T2,524288.0,28518.40,2020-11-13,ibm
518915,761,SIGN_19DC1_T2M_BACKUP_01,SDC1_T2_FCM,T2_FCM,1048576.0,967772.16,2020-11-13,ibm


In [116]:
# création d'une colonne pourcentage_used 
#df_ibm_volume_reduit['pourcentage_used'] = df_ibm_volume_reduit.used_capacity/df_ibm_volume_reduit.capacity*100
#df_ibm_volume_reduit

In [117]:
# fonction qui convertit en GB
def en_giga(x):
    return x/1024

In [118]:
# transformation des colonnes capacity, used_capacity,real_capacity, free_capacity, uncompressed_used_capacity, used_capacity_before_reduction

df_ibm_volume_GB = df_ibm_volume_reduit.copy()
for colonne in ["capacity", "used_capacity"]:
     df_ibm_volume_GB[colonne] = df_ibm_volume_reduit[colonne].map(lambda x : en_giga(x)).copy()
df_ibm_volume_GB

,ID,name,pool,offre,capacity,used_capacity,date,origine
0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC3_T0,T0,10.0,9.83,2018-10-03,ibm
1,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC1_T0,T0,10.0,9.83,2018-10-03,ibm
2,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,SDC1_T2,T2,200.0,44.46,2018-10-03,ibm
3,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,SDC1_T2,T2,200.0,44.47,2018-10-03,ibm
4,3,INFS_19DC1_T2M_TEST_EXLIDAT1_01,SDC3_T2,T2,10.0,10.53,2018-10-03,ibm
...,...,...,...,...,...,...,...,...
518912,753,PRVN_19DC1_T2M_PRVN_01_DS_01,SDC3_T2,T2,512.0,86.44,2020-11-13,ibm
518913,754,PRVN_19DC3_T2M_PRVN_01_DS_01,SDC3_T2,T2,512.0,27.94,2020-11-13,ibm
518914,754,PRVN_19DC3_T2M_PRVN_01_DS_01,SDC1_T2,T2,512.0,27.85,2020-11-13,ibm
518915,761,SIGN_19DC1_T2M_BACKUP_01,SDC1_T2_FCM,T2_FCM,1024.0,945.09,2020-11-13,ibm


In [119]:
#vérification des valeurs manquantes
df_ibm_volume_GB.isnull().any()

ID               False
name             False
pool             False
offre            False
capacity         False
used_capacity    False
date             False
origine          False
dtype: bool

In [120]:
## Pour avoir la typologie des volumes , on crée à partir d'un fichier IBM server un DF liant le nom du volume
#au server qui l'héberge

In [121]:
# création de la liste des fichiers à récupérer pour IBM server (finalement non utilisée)

basepath = 'C:/Users/lsarlat/Documents/Capacity_planning/CP3/données_version3/ibm/server'
# List all files in a directory using scandir()
liste_ibm_server=[]
with os.scandir(basepath) as entries: # renvoie un itérateur
    for entry in entries:
        if entry.is_file():
            liste_ibm_server.append(str(entry.name))
print(len(liste_ibm_server))

729


In [122]:
# Fonction qui trouve la typologie du server
def typo(serveur):
    if "APP" in serveur:
        typo = "APP"
    elif "WEB" in serveur:
        typo = "WEB"
    elif "DAT" in serveur:
        typo = "DAT"
    elif "SQL" in serveur:
        typo = "DAT"
    elif "ORA" in serveur:
        typo = "DAT"
    elif "VIR" in serveur:
        typo = "VIR"
    elif "vir" in serveur:
        typo = "VIR"
    elif "FIC" in serveur:
        typo = "FIC"
    elif "MES" in serveur:
        typo = "MES"
    elif "DNS" in serveur:
        typo = "DNS"
    else:
        typo = "AUTRE"
 
    return typo

In [123]:
# création du DF Server volmue avec un fichier server IBM (le plus récent) 
# taille du fichier 175 000 ko.

df_ibm_server = pd.DataFrame()
for name in ["20200930_mutsapsan019_lshostvdiskmap.txt"]:
    with open(os.path.join(basepath, name)) as f:
        df_temp = pd.read_csv(f, delim_whitespace =True, names =['ID', 'name',"SCSI_id" ,"vdisk_id" ,"vdisk_name" ,"vdisk_UID","IO_group_id","IO_group_name", "mapping_type","host_cluster_id","host_cluster_name"])
        df_temp['date']= name[:8]
        df_ibm_server = pd.concat([df_ibm_server, df_temp])


# supprime les lignes où le name n'est pas bon.
df_ibm_server = df_ibm_server.drop(df_ibm_server.name[df_ibm_server.name == "name"].index)
df_ibm_server = df_ibm_server.drop(df_ibm_server.name[df_ibm_server.vdisk_name.isnull()].index)

# création d'une colonne typo
df_ibm_server['typologie'] = df_ibm_server.name
df_ibm_server.typologie = df_ibm_server.typologie.map(lambda x : typo(x))
df_ibm_server

,ID,name,SCSI_id,vdisk_id,vdisk_name,vdisk_UID,IO_group_id,IO_group_name,mapping_type,host_cluster_id,host_cluster_name,date,typologie
2,0,EXLIDAT1,0,428,INFN_19DC3_T2M_EXLIDAT1_TEST_01,600507680C80856BD8000000000001D0,0,io_grp0,private,NaN,NaN,20200930,DAT
3,0,EXLIDAT1,3,12,MUTN_19DC1_T2S_BACULA_CATALOGUE_01,600507680C80856BD80000000000000E,0,io_grp0,private,NaN,NaN,20200930,DAT
4,0,EXLIDAT1,1,11,MUTN_19DC1_T2S_BACULA_PGSQL_01,600507680C80856BD80000000000000D,1,io_grp1,private,NaN,NaN,20200930,DAT
7,1,EXLIDAT2,1,10,MUTN_19DC3_T2S_BACULA_PGSQL_01,600507680C80856BD800000000000014,0,io_grp0,private,NaN,NaN,20200930,DAT
8,1,EXLIDAT2,3,9,MUTN_19DC3_T2S_BACULA_CATALOGUE_01,600507680C80856BD800000000000013,1,io_grp1,private,NaN,NaN,20200930,DAT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129240,189,MUTSVPVIR121,3,302,MUTS_19DC1_T2S_MUTS_11_DS_03,600507680C80856BD8000000000007D0,2,io_grp2,shared,46,CL_CLOUD_P_MUTS_11,20200930,VIR
1129241,189,MUTSVPVIR121,4,235,MUTS_19DC1_T0S_MUTS_11_DS_02,600507680C80856BD800000000000800,2,io_grp2,shared,46,CL_CLOUD_P_MUTS_11,20200930,VIR
1129242,189,MUTSVPVIR121,5,639,MUTS_19DC1_T2S_NFSELK_DS_01,600507680C80856BD80000000000082E,2,io_grp2,shared,46,CL_CLOUD_P_MUTS_11,20200930,VIR
1129243,189,MUTSVPVIR121,6,703,MUTS_19DC1_T2S_MUTS_11_DS_04,600507680C80856BD800000000000841,2,io_grp2,shared,46,CL_CLOUD_P_MUTS_11,20200930,VIR


In [124]:
df_ibm_server.typologie.value_counts()

VIR    498054
DAT    460564
WEB        18
Name: typologie, dtype: int64

In [125]:
len(df_ibm_server.name.unique())

226

In [126]:
len(df_ibm_server.vdisk_name.unique())

920

In [127]:
#Création d'un df pour connaitre la typologie du volume et le nom de client(si possible) à partir d'un nom de server associé
liste_volume = []
liste_type = []
liste_client =[]
for volume in df_ibm_server.vdisk_name.unique():
    if volume in liste_volume:
        pass
    else:
        liste_volume.append(volume)
        liste_client.append(df_ibm_server[df_ibm_server.vdisk_name == volume].name.unique()[0][:3])
        liste_type.append(df_ibm_server[df_ibm_server.vdisk_name == volume].typologie.unique()[0])
df_t = pd.DataFrame({"name_v":liste_volume,"client":liste_client,"type":liste_type})
df_t

,name_v,client,type
0,INFN_19DC3_T2M_EXLIDAT1_TEST_01,EXL,DAT
1,MUTN_19DC1_T2S_BACULA_CATALOGUE_01,EXL,DAT
2,MUTN_19DC1_T2S_BACULA_PGSQL_01,EXL,DAT
3,MUTN_19DC3_T2S_BACULA_PGSQL_01,EXL,DAT
4,MUTN_19DC3_T2S_BACULA_CATALOGUE_01,EXL,DAT
...,...,...,...
915,POLS_19DC1_T2S_POLS_01_DS_01,POL,VIR
916,POLS_19DC3_T2S_POLS_01_DS_02,POL,VIR
917,POLS_19DC3_T2S_POLS_01_DS_04,POL,VIR
918,POLS_19DC3_T2M_POLS_01_DS_01,POL,VIR


In [128]:
print(df_t.type.value_counts())

DAT    537
VIR    380
WEB      3
Name: type, dtype: int64


In [129]:
# création du DF Server volmue avec un fichier server IBM (le plus ancien) 
# taille du fichier 43 000 ko.

df_ibm_server_old = pd.DataFrame()
for name in ["20181003_mutsapsan019_lshostvdiskmap.txt"]:
    with open(os.path.join(basepath, name)) as f:
        df_temp = pd.read_csv(f, delim_whitespace =True, names =['ID', 'name',"SCSI_id" ,"vdisk_id" ,"vdisk_name" ,"vdisk_UID","IO_group_id","IO_group_name", "mapping_type","host_cluster_id","host_cluster_name"])
        df_temp['date']= name[:8]
        df_ibm_server_old = pd.concat([df_ibm_server_old, df_temp])


# supprime les lignes où le name n'est pas bon.
df_ibm_server_old = df_ibm_server_old.drop(df_ibm_server_old.name[df_ibm_server_old.name == "name"].index)
df_ibm_server_old = df_ibm_server_old.drop(df_ibm_server_old.name[df_ibm_server_old.vdisk_name.isnull()].index)

# création d'une colonne typo
df_ibm_server_old['typologie'] = df_ibm_server_old.name
df_ibm_server_old.typologie = df_ibm_server_old.typologie.map(lambda x : typo(x))
df_ibm_server_old

,ID,name,SCSI_id,vdisk_id,vdisk_name,vdisk_UID,IO_group_id,IO_group_name,mapping_type,host_cluster_id,host_cluster_name,date,typologie
2,0,EXLIDAT1,0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,600507680C80856BD800000000000000,0,io_grp0,private,NaN,NaN,20181003,DAT
3,0,EXLIDAT1,3,12,MUTN_19DC1_T2S_BACULA_CATALOGUE_01,600507680C80856BD80000000000000E,0,io_grp0,private,NaN,NaN,20181003,DAT
4,0,EXLIDAT1,0,3,INFS_19DC1_T2M_TEST_EXLIDAT1_01,600507680C80856BD800000000000005,1,io_grp1,private,NaN,NaN,20181003,DAT
5,0,EXLIDAT1,1,11,MUTN_19DC1_T2S_BACULA_PGSQL_01,600507680C80856BD80000000000000D,1,io_grp1,private,NaN,NaN,20181003,DAT
8,1,EXLIDAT2,0,6,INFS_19DC3_T2M_TEST_EXLIDAT2_01,600507680C80856BD800000000000008,0,io_grp0,private,NaN,NaN,20181003,DAT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,89,IPSSVPVIR016,2,236,IPSS_19DC3_T2M_DS_03,600507680C80856BD8000000000000FE,0,io_grp0,shared,11,CL_CLOUD_P_IPSS_02,20181003,VIR
2223,89,IPSSVPVIR016,0,231,IPSS_19DC1_T2M_DS_01,600507680C80856BD8000000000000F9,1,io_grp1,shared,11,CL_CLOUD_P_IPSS_02,20181003,VIR
2224,89,IPSSVPVIR016,1,233,IPSS_19DC1_T2M_DS_03,600507680C80856BD8000000000000FB,1,io_grp1,shared,11,CL_CLOUD_P_IPSS_02,20181003,VIR
2225,89,IPSSVPVIR016,2,235,IPSS_19DC3_T2M_DS_02,600507680C80856BD8000000000000FD,1,io_grp1,shared,11,CL_CLOUD_P_IPSS_02,20181003,VIR


In [130]:
#Création d'un df pour connaitre la typologie du volume et le nom de client(si possible) à partir d'un nom de server associé
# idem mais avec le fichier le plus ancien.
liste_volume_old = []
liste_type_old = []
liste_client_old =[]
for volume in df_ibm_server_old.vdisk_name.unique():
    if volume in liste_volume_old:
        pass
    else:
        liste_volume_old.append(volume)
        liste_client_old.append(df_ibm_server_old[df_ibm_server_old.vdisk_name == volume].name.unique()[0][:3])
        liste_type_old.append(df_ibm_server_old[df_ibm_server_old.vdisk_name == volume].typologie.unique()[0])
df_t_old = pd.DataFrame({"name_v":liste_volume_old,"client":liste_client_old,"type":liste_type_old})
df_t_old

,name_v,client,type
0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,EXL,DAT
1,MUTN_19DC1_T2S_BACULA_CATALOGUE_01,EXL,DAT
2,INFS_19DC1_T2M_TEST_EXLIDAT1_01,EXL,DAT
3,MUTN_19DC1_T2S_BACULA_PGSQL_01,EXL,DAT
4,INFS_19DC3_T2M_TEST_EXLIDAT2_01,EXL,DAT
...,...,...,...
377,LNAS_19DC3_T2S_ORALNAP_DATA_FG2_01,LNA,DAT
378,LNAS_19DC3_T2S_ORALNAP_DATA_FG2_03,LNA,DAT
379,A24N_19DC1_T2S_U01_2,NAL,WEB
380,A24N_19DC1_T2S_U01_1,NAL,WEB


In [131]:
print("nb de client",len(df_t_old.client.unique()))
# nb de volumes par clients reconnus
df_t_old.client.value_counts()

nb de client 14


MUT    114
ELE    106
USG     41
LNA     26
PXP     22
CMC     22
INF     20
EXL      8
IPS      7
ORA      5
HAR      4
NAL      3
GAM      3
POL      1
Name: client, dtype: int64

In [132]:
compt=0
for v in df_t.name_v:
    if v not in df_t_old.name_v.to_list():
        compt+=1
print("nb de nouveaux volumes ",compt)
compt=0
for v in df_t_old.name_v:
    if v not in df_t.name_v.to_list():
        compt+=1
print("nb d'anciens volumes qui auraient disparus",compt)

nb de nouveaux volumes  584
nb d'anciens volumes qui auraient disparus 46


In [133]:
# test pour voir si la typologie d'un volume change entre le fichier le plus ancien et le plus récent.
#for i in range (len(df_t_old)):
 #   if df_t_old.iloc[i].type != df_t_old.iloc[i].new_typo:
  #      print(i)

In [134]:
liste_non_typé=[]
for v in df_ibm_volume_GB.name.unique():
    if v not in df_t[df_t.name_v == v]:
        liste_non_typé.append(v)
print('nb de volumes distincts',len(df_ibm_volume_GB.name.unique()))
print('nb de volumes typés',len(df_ibm_volume_GB.name.unique())-len(liste_non_typé))
print('nb de volumes non typés',len(liste_non_typé))

nb de volumes distincts 1249
nb de volumes typés 0
nb de volumes non typés 1249


In [135]:
#non_reconn_client=[]
#for v in df_ibm_volume_GB.name.unique():
#    if v not in df_t.name_v:
#        non_reconn_client.append(v[:3])
#print('nb de volumes',len(df_ibm_volume_GB.name.unique()))
# print('nb de volumes typés',len(df_ibm_volume_GB.name.unique())-len(non_reconn_client))
# print('nb de volumes non typés',len(non_reconn_client))

In [136]:
# Etiquetage en typologie et nom des clients des volumes ibm à partir du dataframe constitué précédemment df_t.
# Si le serveur n'est reconnu, on tente de trouver la typo via son nom avec la fonction typo()
df_ibm_volume_plus = df_ibm_volume_GB.copy()

liste_typo =[]
liste_client=[]
for v in df_ibm_volume_plus.name:
    if v in df_t.name_v.unique().tolist(): # récupération de la typologie via le df df_t
        liste_typo.append(df_t[df_t.name_v == v].type.values[0])
        liste_client.append(df_t[df_t.name_v == v].client.values[0])
    else:
        if v[:3]=="INF":
            liste_typo.append(typo(v))   # appel de la fonction typo() pour trouver la typologie
            liste_client.append('INF')
        elif v[:3]=="SIG":
            liste_typo.append(typo(v))   
            liste_client.append('SIG')
        else :
            liste_typo.append(typo(v)) 
            liste_client.append('Client_non_enregistré')
df_ibm_volume_plus['typologie'] = liste_typo
df_ibm_volume_plus['client'] = liste_client
df_ibm_volume_plus

,ID,name,pool,offre,capacity,used_capacity,date,origine,typologie,client
0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC3_T0,T0,10.0,9.83,2018-10-03,ibm,DAT,INF
1,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC1_T0,T0,10.0,9.83,2018-10-03,ibm,DAT,INF
2,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,SDC1_T2,T2,200.0,44.46,2018-10-03,ibm,DAT,CMC
3,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,SDC1_T2,T2,200.0,44.47,2018-10-03,ibm,DAT,CMC
4,3,INFS_19DC1_T2M_TEST_EXLIDAT1_01,SDC3_T2,T2,10.0,10.53,2018-10-03,ibm,DAT,INF
...,...,...,...,...,...,...,...,...,...,...
518912,753,PRVN_19DC1_T2M_PRVN_01_DS_01,SDC3_T2,T2,512.0,86.44,2020-11-13,ibm,AUTRE,Client_non_enregistré
518913,754,PRVN_19DC3_T2M_PRVN_01_DS_01,SDC3_T2,T2,512.0,27.94,2020-11-13,ibm,AUTRE,Client_non_enregistré
518914,754,PRVN_19DC3_T2M_PRVN_01_DS_01,SDC1_T2,T2,512.0,27.85,2020-11-13,ibm,AUTRE,Client_non_enregistré
518915,761,SIGN_19DC1_T2M_BACKUP_01,SDC1_T2_FCM,T2_FCM,1024.0,945.09,2020-11-13,ibm,AUTRE,SIG


In [137]:
## Enregistrement 
df_ibm_volume_plus.to_csv("df_ibm_volume_plus.csv", index=False)

In [138]:
df_ibm_volume_plus = pd.read_csv("df_ibm_volume_plus.csv")

In [139]:
df_ibm_month = df_ibm_volume_plus.copy()
liste_année_mois = []
for i in df_ibm_month.date:
    liste_année_mois.append(str(i)[:7])
df_ibm_month['année_mois'] =liste_année_mois
df_ibm_month

,ID,name,pool,offre,capacity,used_capacity,date,origine,typologie,client,année_mois
0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC3_T0,T0,10.0,9.83,2018-10-03,ibm,DAT,INF,2018-10
1,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC1_T0,T0,10.0,9.83,2018-10-03,ibm,DAT,INF,2018-10
2,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,SDC1_T2,T2,200.0,44.46,2018-10-03,ibm,DAT,CMC,2018-10
3,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,SDC1_T2,T2,200.0,44.47,2018-10-03,ibm,DAT,CMC,2018-10
4,3,INFS_19DC1_T2M_TEST_EXLIDAT1_01,SDC3_T2,T2,10.0,10.53,2018-10-03,ibm,DAT,INF,2018-10
...,...,...,...,...,...,...,...,...,...,...,...
518912,753,PRVN_19DC1_T2M_PRVN_01_DS_01,SDC3_T2,T2,512.0,86.44,2020-11-13,ibm,AUTRE,Client_non_enregistré,2020-11
518913,754,PRVN_19DC3_T2M_PRVN_01_DS_01,SDC3_T2,T2,512.0,27.94,2020-11-13,ibm,AUTRE,Client_non_enregistré,2020-11
518914,754,PRVN_19DC3_T2M_PRVN_01_DS_01,SDC1_T2,T2,512.0,27.85,2020-11-13,ibm,AUTRE,Client_non_enregistré,2020-11
518915,761,SIGN_19DC1_T2M_BACKUP_01,SDC1_T2_FCM,T2_FCM,1024.0,945.09,2020-11-13,ibm,AUTRE,SIG,2020-11


In [140]:
# création de listes des métriques sur la capacité utilisée des volumes
dates=[]
count = []
mean =[]
std =[]
val_min =[]
val_max =[]
quartile25 =[]
quartile50 =[]
quartile75 =[]
for date in df_ibm_month.année_mois.unique():
    dates.append(date)
    count.append(df_ibm_month[df_ibm_month.année_mois == date].used_capacity.describe()['count'])
    mean.append(df_ibm_month[df_ibm_month.année_mois == date].used_capacity.describe()['mean'])
    std.append(df_ibm_month[df_ibm_month.année_mois == date].used_capacity.describe()['std'])
    val_min.append(df_ibm_month[df_ibm_month.année_mois == date].used_capacity.describe()['min'])
    val_max.append(df_ibm_month[df_ibm_month.année_mois == date].used_capacity.describe()['max'])
    quartile25.append(df_ibm_month[df_ibm_month.année_mois == date].used_capacity.describe()['25%'])
    quartile50.append(df_ibm_month[df_ibm_month.année_mois == date].used_capacity.describe()['50%'])
    quartile75.append(df_ibm_month[df_ibm_month.année_mois == date].used_capacity.describe()['75%'])
    
array = np.array([mean,std,val_min,val_max,quartile25,quartile50, quartile75])
df_month_variables = pd.DataFrame(array, index = ["mean","std","val_min","val_max","quartile25","quartile50", "quartile75"], columns =dates)
df_month_variables

,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,...,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11
mean,185.159506,203.788205,213.668160,214.897010,242.417830,255.580737,266.906876,301.785019,300.516112,323.328316,...,455.175448,461.335543,454.102742,468.175645,476.007071,409.702728,407.103968,458.372155,540.515272,555.674497
std,459.608061,492.090478,515.893756,532.522939,593.906265,618.438390,640.143951,676.525586,682.476855,723.061245,...,892.930879,915.895876,919.204058,940.607896,945.494960,825.757814,826.816601,883.624388,951.906159,971.808345
val_min,0.000156,0.000156,0.000732,0.000156,0.000156,0.000156,0.000732,0.000156,0.000156,0.000732,...,0.000156,0.000732,0.000732,0.000732,0.000732,0.000156,0.000732,0.000732,0.000156,0.000732
val_max,2703.360000,2918.400000,3123.200000,3727.360000,4044.800000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,...,5611.520000,5560.320000,5591.040000,5591.040000,5591.040000,5591.040000,5355.520000,5355.520000,4782.080000,5376.000000
quartile25,3.040000,5.197500,5.720000,5.440000,5.440000,2.740000,2.520000,4.200000,1.600000,1.620000,...,6.270000,6.090000,3.515000,3.430000,4.420000,7.230000,6.980000,8.720000,9.170000,10.740000
quartile50,21.210000,22.440000,30.420000,27.770000,30.420000,30.520000,30.510000,32.950000,31.760000,31.850000,...,66.780000,66.340000,59.270000,57.630000,62.080000,74.120000,71.350000,77.880000,84.920000,85.840000
quartile75,89.605000,97.500000,102.645000,99.630000,105.792500,122.940000,133.980000,157.170000,165.530000,180.850000,...,352.997500,338.750000,317.715000,340.410000,357.180000,299.760000,295.590000,357.260000,507.880000,527.880000


In [141]:
df_month_variables = df_month_variables.T

In [142]:
df_month_variables

,mean,std,val_min,val_max,quartile25,quartile50,quartile75
2018-10,185.159506,459.608061,0.000156,2703.36,3.0400,21.21,89.6050
2018-11,203.788205,492.090478,0.000156,2918.40,5.1975,22.44,97.5000
2018-12,213.668160,515.893756,0.000732,3123.20,5.7200,30.42,102.6450
2019-01,214.897010,532.522939,0.000156,3727.36,5.4400,27.77,99.6300
2019-02,242.417830,593.906265,0.000156,4044.80,5.4400,30.42,105.7925
2019-03,255.580737,618.438390,0.000156,4096.00,2.7400,30.52,122.9400
2019-04,266.906876,640.143951,0.000732,4096.00,2.5200,30.51,133.9800
2019-05,301.785019,676.525586,0.000156,4096.00,4.2000,32.95,157.1700
2019-06,300.516112,682.476855,0.000156,4096.00,1.6000,31.76,165.5300
2019-07,323.328316,723.061245,0.000732,4096.00,1.6200,31.85,180.8500


In [143]:
# création de listes des métriques sur la saturation des volumes

dates=[]
count = []
mean =[]
std =[]
val_min =[]
val_max =[]
quartile25 =[]
quartile50 =[]
quartile75 =[]
for date in df_ibm_month.année_mois.unique():
    dates.append(date)
    count.append(df_ibm_month[df_ibm_month.année_mois == date].used_capacity.describe()['count'])
    mean.append(df_ibm_month[df_ibm_month.année_mois == date].used_capacity.describe()['mean'])
    std.append(df_ibm_month[df_ibm_month.année_mois == date].used_capacity.describe()['std'])
    val_min.append(df_ibm_month[df_ibm_month.année_mois == date].used_capacity.describe()['min'])
    val_max.append(df_ibm_month[df_ibm_month.année_mois == date].used_capacity.describe()['max'])
    quartile25.append(df_ibm_month[df_ibm_month.année_mois == date].used_capacity.describe()['25%'])
    quartile50.append(df_ibm_month[df_ibm_month.année_mois == date].used_capacity.describe()['50%'])
    quartile75.append(df_ibm_month[df_ibm_month.année_mois == date].used_capacity.describe()['75%'])
    
array = np.array([mean,std,val_min,val_max,quartile25,quartile50, quartile75])
df_month_variables = pd.DataFrame(array, index = ["mean","std","val_min","val_max","quartile25","quartile50", "quartile75"], columns =dates)
df_month_variables = df_month_variables.T
df_month_variables

,mean,std,val_min,val_max,quartile25,quartile50,quartile75
2018-10,185.159506,459.608061,0.000156,2703.36,3.0400,21.21,89.6050
2018-11,203.788205,492.090478,0.000156,2918.40,5.1975,22.44,97.5000
2018-12,213.668160,515.893756,0.000732,3123.20,5.7200,30.42,102.6450
2019-01,214.897010,532.522939,0.000156,3727.36,5.4400,27.77,99.6300
2019-02,242.417830,593.906265,0.000156,4044.80,5.4400,30.42,105.7925
2019-03,255.580737,618.438390,0.000156,4096.00,2.7400,30.52,122.9400
2019-04,266.906876,640.143951,0.000732,4096.00,2.5200,30.51,133.9800
2019-05,301.785019,676.525586,0.000156,4096.00,4.2000,32.95,157.1700
2019-06,300.516112,682.476855,0.000156,4096.00,1.6000,31.76,165.5300
2019-07,323.328316,723.061245,0.000732,4096.00,1.6200,31.85,180.8500


In [144]:
#A partir des fichiers XLM , création d'un dico avec en clé les pool et en valeur 
#la capacity maximale du pool en GB
maxi ={'SDC1_T0' : 47667,"SDC1_T2" : 56852, "SDC1_T2_FCM":183828, "SDC3_T0":47667,"SDC3_T2":56852, "SDC3_T2_FCM":183838}

used_01_09 = {'SDC1_T0' : 39557,"SDC1_T2" : 43889, "SDC1_T2_FCM":97178, "SDC3_T0":39957,"SDC3_T2":43735, "SDC3_T2_FCM":90501} 


In [145]:
df_ibm_volume_GB[df_ibm_volume_GB.date=="2020-09-01"].groupby('date').used_capacity.sum()

date
2020-09-01    343235.906279
Name: used_capacity, dtype: float64

# 3 PAR

 ## Pour les volumes

In [146]:
# création de la liste des fichiers à récupérer pour 3par volume

basepath_3par_volume = 'C:/Users/lsarlat/Documents/Capacity_planning/CP3/données_version3/3par/volume'
# List all files in a directory using scandir()
liste_3par_volume=[]
with os.scandir(basepath_3par_volume) as entries: # renvoie un itérateur
    for entry in entries:
        if entry.is_file():
            liste_3par_volume.append(str(entry.name))
print(len(liste_3par_volume))

2002


In [167]:
# création du DF principal à partir de la liste_3par_volume

df_3par_volume = pd.DataFrame()
for name in liste_3par_volume:
    with open(os.path.join(basepath_3par_volume, name)) as f:
        
        # gestion des formats de fichiers avant / après le 08 janvier 2020
        if datetime.datetime.strptime(str(int(name[:8])),'%Y%m%d')<= datetime.datetime(2020,1,8):
            df_temp = pd.read_csv(f,delim_whitespace =True,  names= ["ID","name","Prov","Type","adm_Rsvd","adm_Used","snp_Rsvd","snp_Used","%_snp_Used","snp_Wrn","snp_Lim","usr_Rsvd" ,"usr_Used","%usr_Used", "usr_Wrn" ,"usr_Lim","Tot_Rsvd","VSize","Compaction","Dedup"], header = 4)
            df_temp['date']= datetime.datetime.strptime(str(int(name[:8])),'%Y%m%d')
            df_temp['pool']= name[9:19]
            df_temp = df_temp.rename(columns={"Tot_Rsvd": "Tot_used"})
        else:
            df_temp = pd.read_csv(f,delim_whitespace =True,  names= ["ID","name","Prov","Type","adm_Rsvd","adm_Used","snp_Rsvd","snp_Used","%_snp_Used","snp_Wrn","snp_Lim","usr_Rsvd" ,"usr_Used","%usr_Used", "usr_Wrn" ,"usr_Lim","Tot_Rsvd","Tot_used","HostWR","VSize","Compact","Compress"], header = 4)
            df_temp['date']= datetime.datetime.strptime(str(int(name[:8])),'%Y%m%d')
            df_temp['pool']= name[9:19]
        df_3par_volume = pd.concat([df_3par_volume, df_temp])
df_3par_volume

,ID,name,Prov,Type,adm_Rsvd,adm_Used,snp_Rsvd,snp_Used,%_snp_Used,snp_Wrn,...,Tot_used,VSize,Compaction,Dedup,date,pool,Tot_Rsvd,HostWR,Compact,Compress
0,70,GTGN_05T2N_DS_GTG_DATA_02,tpvv,base,1664,1418,0.0,0,0,--,...,2301696.0,2301952.0,1,--,2018-02-11,mutapsan05,NaN,NaN,NaN,NaN
1,69,GTGN_05T2N_DS_GTG_OS_02,tpvv,base,256,51,512.0,0,0,0,...,73984.0,512000.0,7.5,--,2018-02-11,mutapsan05,NaN,NaN,NaN,NaN
2,73,IAXI_05T2S_OVM-12CR2_01,tpvv,base,256,21,512.0,0,0,0,...,32000.0,30720.0,1,--,2018-02-11,mutapsan05,NaN,NaN,NaN,NaN
3,44,IAXI_05T2S_OVM-RAC01_01,tpvv,base,256,15,0.0,0,0,--,...,21248.0,25600.0,1.3,--,2018-02-11,mutapsan05,NaN,NaN,NaN,NaN
4,45,IAXI_05T2S_OVM-RAC01_02,tpvv,base,256,15,0.0,0,0,--,...,21248.0,25600.0,1.3,--,2018-02-11,mutapsan05,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,68,SNIN_06T2S_ORASNI_FRA_FG2_01,tpvv,No,No,base,512.0,0,0,0,...,10801.0,20480.0,NaN,NaN,2020-11-12,mutapsan06,15360.0,10800.0,1.90,--
23,66,SNIN_06T2S_ORASNI_OCR_01_SDC3,tpvv,No,No,base,512.0,0,0,0,...,9.0,512.0,NaN,NaN,2020-11-12,mutapsan06,1536.0,8.0,>25,--
24,67,SNIN_06T2S_ORASNI_VOT_01_SDC3,tpvv,No,No,base,512.0,0,0,0,...,1.0,512.0,NaN,NaN,2020-11-12,mutapsan06,1536.0,0.0,>25,--
25,----------------------------------------------...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2020-11-12,mutapsan06,NaN,NaN,NaN,NaN


In [168]:
# supprime les lignes où le name n'est pas bon.
df_3par_volume = df_3par_volume.drop(df_3par_volume.name[df_3par_volume.name.isnull()].index)
df_3par_volume = df_3par_volume.drop(df_3par_volume.name[df_3par_volume.name=='total'].index)
df_3par_volume = df_3par_volume[df_3par_volume.name !="No"]
df_3par_volume

,ID,name,Prov,Type,adm_Rsvd,adm_Used,snp_Rsvd,snp_Used,%_snp_Used,snp_Wrn,...,Tot_used,VSize,Compaction,Dedup,date,pool,Tot_Rsvd,HostWR,Compact,Compress
0,70,GTGN_05T2N_DS_GTG_DATA_02,tpvv,base,1664,1418,0.0,0,0,--,...,2301696.0,2301952.0,1,--,2018-02-11,mutapsan05,NaN,NaN,NaN,NaN
1,69,GTGN_05T2N_DS_GTG_OS_02,tpvv,base,256,51,512.0,0,0,0,...,73984.0,512000.0,7.5,--,2018-02-11,mutapsan05,NaN,NaN,NaN,NaN
2,73,IAXI_05T2S_OVM-12CR2_01,tpvv,base,256,21,512.0,0,0,0,...,32000.0,30720.0,1,--,2018-02-11,mutapsan05,NaN,NaN,NaN,NaN
3,44,IAXI_05T2S_OVM-RAC01_01,tpvv,base,256,15,0.0,0,0,--,...,21248.0,25600.0,1.3,--,2018-02-11,mutapsan05,NaN,NaN,NaN,NaN
4,45,IAXI_05T2S_OVM-RAC01_02,tpvv,base,256,15,0.0,0,0,--,...,21248.0,25600.0,1.3,--,2018-02-11,mutapsan05,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,79,SIGN_06DC3_T2S_SIGS_01_DS_02,tpvv,No,No,base,512.0,0,0,0,...,4297477.0,5242880.0,NaN,NaN,2020-11-12,mutapsan06,4303360.0,4297476.0,1.22,--
17,73,SNIN_06T2S_ORASNI_BACKUP_01,tpvv,No,No,base,512.0,0,0,0,...,43975.0,46080.0,NaN,NaN,2020-11-12,mutapsan06,47232.0,43974.0,1.05,--
18,69,SNIN_06T2S_ORASNI_DATA_FG2_01,tpvv,No,No,base,512.0,0,0,0,...,173116.0,204800.0,NaN,NaN,2020-11-12,mutapsan06,175104.0,173115.0,1.18,--
19,70,SNIN_06T2S_ORASNI_DATA_FG2_02,tpvv,No,No,base,512.0,0,0,0,...,174128.0,204800.0,NaN,NaN,2020-11-12,mutapsan06,177152.0,174127.0,1.18,--


In [169]:
# renommage des colonnes
df_3par_volume = df_3par_volume.rename(columns={"Tot_used": "used_capacity", "VSize":"capacity"})

# limitation du nb de colonnes
df_3par_volume = df_3par_volume[["ID","name","capacity","used_capacity","date","pool"]]
df_3par_volume['offre'] = np.array('T2')
#suppression des lignes où la used_capcity est null
df_3par_volume = df_3par_volume[df_3par_volume.used_capacity.notnull()]

# réinitialise les index
df_3par_volume.reset_index(drop=True,inplace =True)

# traitement sur la colonne date
df_3par_volume.date = df_3par_volume.date.map(lambda x :  datetime.datetime.strptime(str(x)[:10],'%Y-%m-%d'))

# ajout d'une colonne origine
df_3par_volume['origine'] = np.array("3par")
df_3par_volume

,ID,name,capacity,used_capacity,date,pool,offre,origine
0,70,GTGN_05T2N_DS_GTG_DATA_02,2301952.0,2301696.0,2018-02-11,mutapsan05,T2,3par
1,69,GTGN_05T2N_DS_GTG_OS_02,512000.0,73984.0,2018-02-11,mutapsan05,T2,3par
2,73,IAXI_05T2S_OVM-12CR2_01,30720.0,32000.0,2018-02-11,mutapsan05,T2,3par
3,44,IAXI_05T2S_OVM-RAC01_01,25600.0,21248.0,2018-02-11,mutapsan05,T2,3par
4,45,IAXI_05T2S_OVM-RAC01_02,25600.0,21248.0,2018-02-11,mutapsan05,T2,3par
...,...,...,...,...,...,...,...,...
42334,79,SIGN_06DC3_T2S_SIGS_01_DS_02,5242880.0,4297477.0,2020-11-12,mutapsan06,T2,3par
42335,73,SNIN_06T2S_ORASNI_BACKUP_01,46080.0,43975.0,2020-11-12,mutapsan06,T2,3par
42336,69,SNIN_06T2S_ORASNI_DATA_FG2_01,204800.0,173116.0,2020-11-12,mutapsan06,T2,3par
42337,70,SNIN_06T2S_ORASNI_DATA_FG2_02,204800.0,174128.0,2020-11-12,mutapsan06,T2,3par


In [170]:
# ajout de colonne Client
df_3par_volume['client'] = df_3par_volume.name
df_3par_volume.client = df_3par_volume.client.map(lambda x : x[:3])
# ajout d'une colonne pourcentage de saturation
#df_3par_volume['pourcentage_used'] = df_3par_volume.used_capacity/df_3par_volume.capacity*100
df_3par_volume

,ID,name,capacity,used_capacity,date,pool,offre,origine,client
0,70,GTGN_05T2N_DS_GTG_DATA_02,2301952.0,2301696.0,2018-02-11,mutapsan05,T2,3par,GTG
1,69,GTGN_05T2N_DS_GTG_OS_02,512000.0,73984.0,2018-02-11,mutapsan05,T2,3par,GTG
2,73,IAXI_05T2S_OVM-12CR2_01,30720.0,32000.0,2018-02-11,mutapsan05,T2,3par,IAX
3,44,IAXI_05T2S_OVM-RAC01_01,25600.0,21248.0,2018-02-11,mutapsan05,T2,3par,IAX
4,45,IAXI_05T2S_OVM-RAC01_02,25600.0,21248.0,2018-02-11,mutapsan05,T2,3par,IAX
...,...,...,...,...,...,...,...,...,...
42334,79,SIGN_06DC3_T2S_SIGS_01_DS_02,5242880.0,4297477.0,2020-11-12,mutapsan06,T2,3par,SIG
42335,73,SNIN_06T2S_ORASNI_BACKUP_01,46080.0,43975.0,2020-11-12,mutapsan06,T2,3par,SNI
42336,69,SNIN_06T2S_ORASNI_DATA_FG2_01,204800.0,173116.0,2020-11-12,mutapsan06,T2,3par,SNI
42337,70,SNIN_06T2S_ORASNI_DATA_FG2_02,204800.0,174128.0,2020-11-12,mutapsan06,T2,3par,SNI


In [171]:
# conversion en GB
df_3par_volume_GB = df_3par_volume.copy()
for colonne in ['capacity','used_capacity']:
    df_3par_volume_GB[colonne] = df_3par_volume_GB[colonne].map(lambda x :en_giga(x))
df_3par_volume_GB

,ID,name,capacity,used_capacity,date,pool,offre,origine,client
0,70,GTGN_05T2N_DS_GTG_DATA_02,2248.0,2247.750000,2018-02-11,mutapsan05,T2,3par,GTG
1,69,GTGN_05T2N_DS_GTG_OS_02,500.0,72.250000,2018-02-11,mutapsan05,T2,3par,GTG
2,73,IAXI_05T2S_OVM-12CR2_01,30.0,31.250000,2018-02-11,mutapsan05,T2,3par,IAX
3,44,IAXI_05T2S_OVM-RAC01_01,25.0,20.750000,2018-02-11,mutapsan05,T2,3par,IAX
4,45,IAXI_05T2S_OVM-RAC01_02,25.0,20.750000,2018-02-11,mutapsan05,T2,3par,IAX
...,...,...,...,...,...,...,...,...,...
42334,79,SIGN_06DC3_T2S_SIGS_01_DS_02,5120.0,4196.754883,2020-11-12,mutapsan06,T2,3par,SIG
42335,73,SNIN_06T2S_ORASNI_BACKUP_01,45.0,42.944336,2020-11-12,mutapsan06,T2,3par,SNI
42336,69,SNIN_06T2S_ORASNI_DATA_FG2_01,200.0,169.058594,2020-11-12,mutapsan06,T2,3par,SNI
42337,70,SNIN_06T2S_ORASNI_DATA_FG2_02,200.0,170.046875,2020-11-12,mutapsan06,T2,3par,SNI


In [172]:
## Enregistrement en csv
df_3par_volume_GB.to_csv("df_3par_volume_GB.csv",index=False)

In [173]:
list_pool_3par = df_3par_volume_GB.pool.unique()

In [174]:
#TEst pour voir si les volumes sont bien indépendants entre IBM et 3par
#for i in df_3par_volume_GB.name:
 #   if i in df_ibm_volume_GB.name.to_list():
  #      print(i)
        
## Pas de doublons. Les volumes IBM ne sont pas dans 3par.

In [175]:
# capacité utilisée par les volumes 3par rn date du  28/08/2020
df_3par_volume_GB[df_3par_volume_GB.date=="2020-08-28"].used_capacity.sum()

60381.8173828125

#  3par server

In [176]:
# création du DF Server volume avec un fichier server 3par (le plus récent 20191027) 

basepath_3par_server = 'C:/Users/lsarlat/Documents/Capacity_planning/CP3/3par/server/lun'
df_3par_server = pd.DataFrame()
for name in ["20191027_mutapsan06_showvlun-lvw.txt"]:
    with open(os.path.join(basepath_3par_server, name)) as f:
        df_temp = pd.read_csv(f, delim_whitespace =True, names =['ID_volume', 'name_volume',"vv_wwn" ,"server" ,"Host_WWN/ISCSI" ,"port","Type","Status", "id"])
        df_temp['date']= name[:8]
        df_3par_server = pd.concat([df_3par_server, df_temp])
        
# supprime les lignes où le name n'est pas bon.
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.name_volume == "VVName"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.name_volume == "VLUNs"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.ID_volume == "VLUN"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.name_volume == "total"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server['Host_WWN/ISCSI'] == "----------------"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.name_volume.isnull()].index)
df_3par_server

,ID_volume,name_volume,vv_wwn,server,Host_WWN/ISCSI,port,Type,Status,id,date
2,0,GRCS_06DC3_T2S_GRCS_00_DS_01,60002AC0000000000000006900019826,GRCSVPVIR100,10008666E8200296,0:0:1,host,set,active,20191027
3,0,GRCS_06DC3_T2S_GRCS_00_DS_01,60002AC0000000000000006900019826,GRCSVPVIR100,10008666E820029A,0:0:2,host,set,active,20191027
4,0,GRCS_06DC3_T2S_GRCS_00_DS_01,60002AC0000000000000006900019826,GRCSVPVIR100,10008666E8200296,1:0:1,host,set,active,20191027
5,0,GRCS_06DC3_T2S_GRCS_00_DS_01,60002AC0000000000000006900019826,GRCSVPVIR100,10008666E820029A,1:0:2,host,set,active,20191027
6,0,GRCS_06DC3_T2S_GRCS_00_DS_01,60002AC0000000000000006900019826,GRCSVPVIR102,10008666E82002A0,0:0:1,host,set,active,20191027
...,...,...,...,...,...,...,...,...,...,...
869,1,SIGN_06DC3_T2S_SIGS_01_DS_02,60002AC0000000000000004F00019826,SIGVPVIR14,21000024FF9148A7,1:0:2,host,set,active,20191027
870,1,IPSS_06T2S_SQL_02,60002AC0000000000000003D00019826,SVR-APP0B,10008666E8200082,0:0:1,host,active,1,20191027
871,1,IPSS_06T2S_SQL_02,60002AC0000000000000003D00019826,SVR-APP0B,10008666E8200080,0:0:2,host,active,1,20191027
872,1,IPSS_06T2S_SQL_02,60002AC0000000000000003D00019826,SVR-APP0B,10008666E8200082,1:0:1,host,active,1,20191027


In [177]:
df_3par_server['typologie'] = df_3par_server.server
df_3par_server.typologie = df_3par_server.typologie.map(lambda x : typo(x))

In [178]:
# étiquetage de la typologie des volumes 3par
df_3par_volume_plus = df_3par_volume_GB.copy()
liste_typo =[]
for v in df_3par_volume_plus.name:
    if v in df_3par_server.name_volume.unique().tolist():
        liste_typo.append(df_3par_server[df_3par_server.name_volume == v].typologie.values[0])
    else:
        liste_typo.append(typo(v))
        
df_3par_volume_plus['typologie'] = liste_typo
df_3par_volume_plus

,ID,name,capacity,used_capacity,date,pool,offre,origine,client,typologie
0,70,GTGN_05T2N_DS_GTG_DATA_02,2248.0,2247.750000,2018-02-11,mutapsan05,T2,3par,GTG,DAT
1,69,GTGN_05T2N_DS_GTG_OS_02,500.0,72.250000,2018-02-11,mutapsan05,T2,3par,GTG,AUTRE
2,73,IAXI_05T2S_OVM-12CR2_01,30.0,31.250000,2018-02-11,mutapsan05,T2,3par,IAX,AUTRE
3,44,IAXI_05T2S_OVM-RAC01_01,25.0,20.750000,2018-02-11,mutapsan05,T2,3par,IAX,AUTRE
4,45,IAXI_05T2S_OVM-RAC01_02,25.0,20.750000,2018-02-11,mutapsan05,T2,3par,IAX,AUTRE
...,...,...,...,...,...,...,...,...,...,...
42334,79,SIGN_06DC3_T2S_SIGS_01_DS_02,5120.0,4196.754883,2020-11-12,mutapsan06,T2,3par,SIG,VIR
42335,73,SNIN_06T2S_ORASNI_BACKUP_01,45.0,42.944336,2020-11-12,mutapsan06,T2,3par,SNI,DAT
42336,69,SNIN_06T2S_ORASNI_DATA_FG2_01,200.0,169.058594,2020-11-12,mutapsan06,T2,3par,SNI,DAT
42337,70,SNIN_06T2S_ORASNI_DATA_FG2_02,200.0,170.046875,2020-11-12,mutapsan06,T2,3par,SNI,DAT


In [179]:
## Enregistrement en csv
df_3par_volume_plus.to_csv("df_3par_volume_plus.csv", index=False)

In [180]:
len(df_3par_volume_plus.name.unique())

140

## Fusion des df volume de IBM et 3par

In [181]:
# union des df ibm et 3PAR
df_volume = pd.concat([df_ibm_volume_plus,df_3par_volume_plus])
df_volume.date = df_volume.date.map(lambda x :  datetime.datetime.strptime(str(x)[:10],'%Y-%m-%d'))
df_volume

,ID,name,pool,offre,capacity,used_capacity,date,origine,typologie,client
0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC3_T0,T0,10.0,9.830000,2018-10-03,ibm,DAT,INF
1,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,SDC1_T0,T0,10.0,9.830000,2018-10-03,ibm,DAT,INF
2,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,SDC1_T2,T2,200.0,44.460000,2018-10-03,ibm,DAT,CMC
3,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,SDC1_T2,T2,200.0,44.470000,2018-10-03,ibm,DAT,CMC
4,3,INFS_19DC1_T2M_TEST_EXLIDAT1_01,SDC3_T2,T2,10.0,10.530000,2018-10-03,ibm,DAT,INF
...,...,...,...,...,...,...,...,...,...,...
42334,79,SIGN_06DC3_T2S_SIGS_01_DS_02,mutapsan06,T2,5120.0,4196.754883,2020-11-12,3par,VIR,SIG
42335,73,SNIN_06T2S_ORASNI_BACKUP_01,mutapsan06,T2,45.0,42.944336,2020-11-12,3par,DAT,SNI
42336,69,SNIN_06T2S_ORASNI_DATA_FG2_01,mutapsan06,T2,200.0,169.058594,2020-11-12,3par,DAT,SNI
42337,70,SNIN_06T2S_ORASNI_DATA_FG2_02,mutapsan06,T2,200.0,170.046875,2020-11-12,3par,DAT,SNI


In [182]:
## Enregistrement en csv
df_volume.to_csv("df_volume.csv", index=False)

In [163]:
df_fichiers_stockage_ibm = pd.DataFrame({'nom_fichier':liste_ibm_volume,'plateforme':np.array('IBM')})
df_fichiers_stockage_3par = pd.DataFrame({'nom_fichier':liste_3par_volume,'plateforme':np.array('3par')})
df_fichiers_stockage = pd.concat([df_fichiers_stockage_ibm,df_fichiers_stockage_3par])
df_fichiers_stockage

,nom_fichier,plateforme
0,20181003_mutsapsan019_lssevdiskcopy.txt,IBM
1,20181004_mutsapsan019_lssevdiskcopy.txt,IBM
2,20181005_mutsapsan019_lssevdiskcopy.txt,IBM
3,20181006_mutsapsan019_lssevdiskcopy.txt,IBM
4,20181007_mutsapsan019_lssevdiskcopy.txt,IBM
...,...,...
1997,20201110_mutapsan06_showvv-space.txt,3par
1998,20201111_mutapsan05_showvv-space.txt,3par
1999,20201111_mutapsan06_showvv-space.txt,3par
2000,20201112_mutapsan05_showvv-space.txt,3par


In [164]:
## Enregistrement en csv
df_fichiers_stockage.to_csv("df_fichiers_stockage.csv", index=False)